## CNN Training Loop

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120) # Display options for o/p
torch.set_grad_enabled(True) # Already on by default

In [2]:
print(torch.__version__)
print(torchvision.__version__)

1.10.0+cu111
0.11.1+cu111


In [3]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [4]:
class Network(nn.Module):

    def __init__(self):

        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):

        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)

        return t

In [5]:
train_set = torchvision.datasets.FashionMNIST(
    root = './content/FashionMNIST',
    train=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ]),
    download=True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./content/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./content/FashionMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./content/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./content/FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./content/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./content/FashionMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./content/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./content/FashionMNIST/FashionMNIST/raw



## Training with all batches: A single epoch

In [6]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

total_loss = 0
total_correct = 0

for batch in train_loader: # Get Batch
    images, labels = batch

    preds = network(images) # Pass Batch
    loss = F.cross_entropy(preds, labels) # Calculate Loss

    optimizer.zero_grad() # zeros the gradients (as gradients are accumulated)

    loss.backward() # Calculate Gradients
    optimizer.step() # Update Weights

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)

print(f"epoch: {0} | total_correct: {total_correct} | loss: {total_loss}")

epoch: 0 | total_correct: 45556 | loss: 377.34258526563644


In [7]:
total_correct / len(train_set)

0.7592666666666666

## Training with multiple epochs: The complete training loop

In [8]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(5):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch

        preds = network(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad() # zeros the gradients (as gradients are accumulated)
        
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(f"epoch: {epoch} | total_correct: {total_correct} | loss: {total_loss}")

epoch: 0 | total_correct: 46505 | loss: 362.2256358861923
epoch: 1 | total_correct: 51051 | loss: 242.91533467173576
epoch: 2 | total_correct: 51719 | loss: 222.01487329602242
epoch: 3 | total_correct: 52157 | loss: 209.10147216916084
epoch: 4 | total_correct: 52471 | loss: 201.34590265154839


In [9]:
total_correct / len(train_set)

0.8745166666666667